In [1]:
import SimpleITK as sitk
import nibabel as nib
import os
import time
import numpy as np
import sys
import warnings

warnings.filterwarnings('ignore')
sys.path.append("../")
from deepneuroan.data_generator import DataGenerator
from deepneuroan.models import ChannelwiseConv3D
from deepneuroan.models import rigid_concatenated

In [6]:
data_dir = "/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/"
template_filepath = os.path.join(data_dir, "template_on_grid")

list_files = []
list_files_tmp = set([])
for root, _, files in os.walk(data_dir):
    for file in files:
        filepath = os.path.join(root, file).split('.')[0]
        if os.path.exists(filepath + ".txt"):
            list_files_tmp.add(filepath)
list_files = list(list_files_tmp)

bs = 1
ncpu = 1
np.random.seed(0)
params_gen = dict(list_files=list_files, template_file=template_filepath, batch_size=bs, avail_cores=ncpu)
all_gen = DataGenerator(partition="all", **params_gen)

In [7]:
with open("/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/rigid_concatenated.json", "r") as json_file:
    model = tf.keras.models.model_from_json(json_file.read(), custom_objects={'ChannelwiseConv3D': ChannelwiseConv3D})
model.load_weights("/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/model_bs16_lr0.01_2019-12-12_10:10:04.h5", by_name=False)
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01)
                  , loss=tf.keras.losses.mean_squared_error
                  , metrics=["mae"])

In [17]:
for i in range(all_gen.__len__()):
    print(all_gen.get_files_batch(i))
    error = model.predict(x=all_gen.__getitem__(i)[0], use_multiprocessing=False, verbose=0) - all_gen.__getitem__(i)[1]
    print("current : ", all_gen.__getitem__(i)[1])
    print("ae : ", error)
    print("mae : ", model.evaluate(x=all_gen.__getitem__(i)[0], y=all_gen.__getitem__(i)[1], batch_size=bs, use_multiprocessing=False, verbose=0)[1])

['/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-202_transfo-000002']
current :  [[ 0.021478  0.236571  0.243625 -0.94033  -0.725687 -1.267853  1.380169]]
ae :  [[ 0.38195992 -0.258983    0.25626796  1.2763      0.846229    0.85686064
   0.22063828]]
mae :  0.5853198
['/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-202_transfo-000007']
current :  [[ 0.940106 -0.191497 -0.649573 -0.078666  1.664281 -0.407973 -0.870109]]
ae :  [[-0.23970908  0.1838034   0.2204777   0.06258649 -1.9983807   0.7531328
   0.9947016 ]]
mae :  0.6361131
['/home/ltetrel/Documents/data/neuromod/derivatives/deepneuroan/training/generated_data/ses-vid001_task-video_run-01_bold_vol-202_transfo-000005']
current :  [[ 0.582358 -0.749234  0.453074 -1.369968  0.367358  0.356667  3.865819]]
ae :  [[-0.05909014  0.48220924 -0.49004593  0.4577881  -0.58346736  0.23217788

In [27]:
np.linalg.norm(all_gen.__getitem__(i)[1][0][:4])

1.734801

In [20]:
np.linalg.norm([ 0.0225887, 0.00184996, -0.00810505, 0.99971 ])

0.9999997238275589

In [25]:
all_gen.__getitem__(i)[1][0][:4]

array([ 0.574592, -1.060401, -0.279431,  1.215256], dtype=float32)

In [ ]:
#http://itk-insight-users.2283740.n2.nabble.com/to-understand-quaternion-versor-and-matrix-in-VersorRigid3DTransform-td2510548.html